#Extract Google Document content
This is a flow to analyze content from collaborative Google Documents used during collaborative workshops. During the workshops, participants vote on policy proposals with emojis, then rewrite them collaboratively. This code will automatically extract the results, count votes, and analyze the rewritten proposals. This code was specifically used for real workshops by the Grand Bargain Project, which discusses policy proposals in six policy areas, each reflected on a document.

The flow will do the following:


1.   Extract text from an aribtrary number of Google Docs
2.   Prase content from tables into structured data
3.   Quantative analysis of the table results to be fed into a Google sheet
4.   NLP sentiment analysis of each proposal.
5.   NLP extraction of topics and themes from each proposal.
6.   Summarizing the propsals with generative AI.



#Extract Google Document content

The first stpe is to extract the content of Google documents.

Requirements:
* A list of the google docs that you plan on extracting content from
* A file with your Google credentials, which you will need to get from Google Cloud. This file is the permsission that allows Jupyter notebooks to access your Google Drive files. For more details, [see this link](https://developers.google.com/workspace/guides/create-credentials).

In [1]:
import requests

# Arbitrary list of Google doc IDs, can be as many as you need
doc_ids = ['ID1', 'ID2', 'ID3']

#This is an example with the specific IDs used fo this project,
#showing an example of how to extract the code for our user case, it will be a
#little different based on the formatting of your Google docs. y
doc_ids = [
    '1UbCOqD0oOfOSfKsuNlUQOR5j_HbjEqr6ho1gGzkkMSM',
    '1UiTEmYYJv6CfQtmribOkvzZTdGWs7GSyHpKHh_5bUvs',
    '1K9-oaLE9bXrFD732yjzp22ySf07Psne03uArWWPSkvU']

links = [f"https://docs.google.com/document/d/{doc_id}" for doc_id in doc_ids]

# Dictionary to store content of each document
documents_content = {}

# Loop through each link to fetch content
for idx, link in enumerate(links, start=1):
    export_url = f'{link}/export?format=txt'
    response = requests.get(export_url)

    if response.status_code == 200:
        content = response.text
        documents_content[f"Issue {idx}"] = content
        print(f"Issue {idx} content fetched successfully.")
    else:
        print(f"Failed to fetch Issue {idx}. Status code: {response.status_code}")

# Example: Access content of Issue 1
print(documents_content["Issue 1"])


Issue 1 content fetched successfully.
Issue 2 content fetched successfully.
Issue 3 content fetched successfully.
﻿Room 1: Economic Mobility
GOAL: Raise workers’ productivity and incomes
Notes: Who are the workers? Should challenge high productivity equal high income? How are we defining economic mobility? How does one assess the effectiveness of community colleges and other training programs? What does a “higher paying job” mean and how does that equate to success? 
Proposal 1
Original statement
	Support
	Don’t support
	Decision
	“Fund community colleges and other training programs[a][b][c] based on how effective they are at building skills that get their graduates into higher paying jobs.”
	

	X
	Revise: write a new one!
	Note: The original statement could negatively impact access to education.
Revision / Addition
	Support
	Don’t support
	Decision
	Revision: Incentivizing community colleges based on career pathways & providing opportunities for credentials
	

	

	Endorse
	Revision: I

In [2]:
### Functions to extract and process document content into a dataframe ###

import re
import pandas as pd

def clean_text(text):
    """
    Removes undesired patterns like '\r\n\t' and extra whitespace.
    """
    if pd.isnull(text):  # Handle NaN or missing values
        return text
    # Remove patterns like '\r\n\t'
    text = re.sub(r'[\r\n\t]+', ' ', text)
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def extract_proposals_v2(text, issue):
    """
    Extracts proposals and voting data from a document text for a specific issue.
    """
    # Patterns to identify sections
    original_pattern = re.compile(
        r'Original statement\s+Support\s+Don’t support\s+Decision\s+“(.*?)”\s+([0-9Xx]*)\s+([0-9Xx]*)\s+(.*?)\s+Revision / Addition',
        re.DOTALL
    )
    revision_pattern = re.compile(
        r'Revision / Addition\s+Support\s+Don’t support\s+Decision\s+((?:.*?\s+[0-9Xx]*\s+[0-9Xx]*\s+(?:Endorse|Reject|N/A)\s*)+)',
        re.DOTALL
    )
    statement_pattern = re.compile(
        r'(.*?)\s+([0-9Xx]*)\s+([0-9Xx]*)\s+(Endorse|Reject|N/A)',
        re.DOTALL
    )

    def count_votes(vote_string):
        """Count votes based on numbers and X/x characters."""
        match = re.search(r'(\d+)', vote_string)  # Match any number
        number_votes = int(match.group(1)) if match else 0
        x_votes = vote_string.lower().count('x')  # Count "X" and "x"
        return number_votes + x_votes

    data = []
    proposals = re.split(r'(Proposal \d+|Additional proposals)', text)

    for i in range(1, len(proposals), 2):
        # Extract proposal title and content
        proposal_title = proposals[i].strip()
        proposal_content = proposals[i + 1].strip()

        # Match original statement
        original_match = original_pattern.search(proposal_content)
        if original_match:
            statement, support, dont_support, decision = original_match.groups()
            data.append({
                "Statement": clean_text(statement.strip()),
                "Statement Type": "Original",
                "Support Votes": count_votes(support),
                "Don’t Support Votes": count_votes(dont_support),
                "Impacted Statement": proposal_title,
                "Decision": decision.strip(),
                "Issue": issue
            })

        # Match revisions/additions
        revision_match = revision_pattern.search(proposal_content)
        if revision_match:
            revision_block = revision_match.group(1).strip()
            revision_statements = statement_pattern.findall(revision_block)

            for statement, support, dont_support, decision in revision_statements:
                if statement.strip():  # Ignore empty or blank statements
                    data.append({
                        "Statement": clean_text(statement.strip()),
                        "Statement Type": "Revision/Addition",
                        "Support Votes": count_votes(support),
                        "Don’t Support Votes": count_votes(dont_support),
                        "Impacted Statement": proposal_title,
                        "Decision": decision.strip(),
                        "Issue": issue
                    })

    return data

# Initialize an empty DataFrame before processing
df = pd.DataFrame()


In [3]:
### Implementing the functions ###

# Process each issue and append results to the DataFrame
for issue, content in documents_content.items():
    # Clean and remove BOM if necessary
    issue = issue.lstrip('\ufeff')  # Remove BOM from issue name
    proposals_data = extract_proposals_v2(content, issue)
    # Concatenate the current data with the existing DataFrame
    df = pd.concat([df, pd.DataFrame(proposals_data)], ignore_index=True)

# Display the combined DataFrame
df.head(30)


,Statement,Statement Type,Support Votes,Don’t Support Votes,Impacted Statement,Decision,Issue
0,Fund community colleges and other training pro...,Original,1,0,Proposal 1,Revise: write a new one!\r\n\tNote: The origin...,Issue 1
1,Revision: Incentivizing community colleges bas...,Revision/Addition,0,0,Proposal 1,Endorse,Issue 1
2,Revision: Incentivize certain programs,Revision/Addition,0,0,Proposal 1,N/A,Issue 1
3,Revision: Separating college & Training programs,Revision/Addition,0,0,Proposal 1,N/A,Issue 1
4,Give lower-paid workers tax credits to increas...,Original,0,0,Proposal 2,Reject\r\n\tNote: This statement is a bit conf...,Issue 1
5,Revision: Remove “increase their incentive to ...,Revision/Addition,0,0,Proposal 2,N/A,Issue 1
6,N/A,Revision/Addition,0,0,Additional proposals,N/A,Issue 1
7,Pay K-12 teachers more to draw top people into...,Original,0,6,Proposal 1,Reject,Issue 2
8,Pay K-12 teachers more.,Revision/Addition,6,0,Proposal 1,Endorse,Issue 2
9,We need to align on a definition for “quality ...,Revision/Addition,0,0,Proposal 1,N/A,Issue 2


In [4]:
### Peaking at the raw text of a document ###

print(documents_content)

{'Issue 1': '\ufeffRoom 1: Economic Mobility\r\nGOAL: Raise workers’ productivity and incomes\r\nNotes: Who are the workers? Should challenge high productivity equal high income? How are we defining economic mobility? How does one assess the effectiveness of community colleges and other training programs? What does a “higher paying job” mean and how does that equate to success? \r\nProposal 1\r\nOriginal statement\r\n\tSupport\r\n\tDon’t support\r\n\tDecision\r\n\t“Fund community colleges and other training programs[a][b][c] based on how effective they are at building skills that get their graduates into higher paying jobs.”\r\n\t\r\n\r\n\tX\r\n\tRevise: write a new one!\r\n\tNote: The original statement could negatively impact access to education.\r\nRevision / Addition\r\n\tSupport\r\n\tDon’t support\r\n\tDecision\r\n\tRevision: Incentivizing community colleges based on career pathways & providing opportunities for credentials\r\n\t\r\n\r\n\t\r\n\r\n\tEndorse\r\n\tRevision: Incentivi

In [5]:
#Function to assign a unique name to the content extracted from each document:
#In this case, it's different policy issues on Economy, Education and Healthcare

def rename_issues(df, issue_column, new_names):
    """
    Replaces 'Issue 1', 'Issue 2', ..., in the specified column with provided names.

    Parameters:
    - df: pandas DataFrame
    - issue_column: str, name of the column with 'Issue N' labels
    - new_names: list of str, new names to replace 'Issue 1', 'Issue 2', etc.

    Returns:
    - df with renamed issues
    """
    issue_map = {f'Issue {i+1}': name for i, name in enumerate(new_names)}
    df[issue_column] = df[issue_column].replace(issue_map)
    return df


In [6]:
### Applying the renaming for our specific Grand Bargain use case ###

new_issue_names = [
    'Economic Mobility',
    'Education',
    'Healthcare',
    ]

df = rename_issues(df, 'Issue', new_issue_names)

In [7]:
# Strip quotation marks from the beginning and end of the 'Statement' column, inspect results
df['Statement'] = df['Statement'].str.strip('\"“”')

df.head()

,Statement,Statement Type,Support Votes,Don’t Support Votes,Impacted Statement,Decision,Issue
0,Fund community colleges and other training pro...,Original,1,0,Proposal 1,Revise: write a new one!\r\n\tNote: The origin...,Economic Mobility
1,Revision: Incentivizing community colleges bas...,Revision/Addition,0,0,Proposal 1,Endorse,Economic Mobility
2,Revision: Incentivize certain programs,Revision/Addition,0,0,Proposal 1,N/A,Economic Mobility
3,Revision: Separating college & Training programs,Revision/Addition,0,0,Proposal 1,N/A,Economic Mobility
4,Give lower-paid workers tax credits to increas...,Original,0,0,Proposal 2,Reject\r\n\tNote: This statement is a bit conf...,Economic Mobility


#NLP Text analysis

In this section, we will use standard NLP libraries to run two kinds of analysis that are prelevant in text processing:

1.    Sentiment analysis: We will examine the positive vs. negative sentiments for each of the policy proposals.
2.    Thematic or topical analysis: We will extract some of the main topics and themes occuring in the policy proposals.

In [9]:
### Define a function to assign sentiment analysis. We are defining a custom spectrum
# of "Positive, Mixed, Negative" using numbered outputs by the Blob library, which range from 1 to -1
# We will reflect both the numerical score and our spectrum category in the output ###

from textblob import TextBlob

# Define a function to get the sentiment score and label
def analyze_sentiment(text):
    blob = TextBlob(text)
    score = blob.sentiment.polarity  # Polarity score between -1 (negative) and +1 (positive)
    if score > 0.2:
        sentiment = "Positive"
    elif score < -0.2:
        sentiment = "Negative"
    else:
        sentiment = "Mixed"
    return score, sentiment

# Apply the function to the 'statement' column
df[['Sentiment Score', 'Sentiment']] = df['Statement'].apply(lambda x: pd.Series(analyze_sentiment(x)))


In [ ]:
#Installing the Spacy library for topical analysis

!pip install spacy
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 66.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [10]:
### Highly contextual -> We will catrgorize policy proposals by whether
# they were in the original proposals, or revised on the Google doc by participants ###

# Add a temporary column to store the original row order
df['Original Order'] = range(len(df))

filtered_rows = []

# Group by both 'Issue' and 'Impacted Statement'
for (issue, proposal), group in df.groupby(['Issue', 'Impacted Statement']):
    if (group['Decision'] == "Endorse").any():  # Check if any row has 'Endorse'
        # Keep all rows with 'Endorse'
        filtered_rows.append(group[group['Decision'] == "Endorse"])
    else:
        # Keep only the 'Original' row
        original_statement = group[group['Statement Type'] == "Original"]
        if not original_statement.empty:
            filtered_rows.append(original_statement)

# Combine the filtered groups into a single DataFrame
df = pd.concat(filtered_rows)

# Sort by the original order and drop the temporary column
df.sort_values('Original Order', inplace=True)
df.drop(columns=['Original Order'], inplace=True)

# Reset index for clarity
df.reset_index(drop=True, inplace=True)

df.head(30)

,Statement,Statement Type,Support Votes,Don’t Support Votes,Impacted Statement,Decision,Issue,Sentiment Score,Sentiment
0,Revision: Incentivizing community colleges bas...,Revision/Addition,0,0,Proposal 1,Endorse,Economic Mobility,0.000000,Mixed
1,Give lower-paid workers tax credits to increas...,Original,0,0,Proposal 2,Reject\r\n\tNote: This statement is a bit conf...,Economic Mobility,0.000000,Mixed
2,Pay K-12 teachers more.,Revision/Addition,6,0,Proposal 1,Endorse,Education,0.500000,Positive
3,"When determining “quality of education,” we sh...",Revision/Addition,6,0,Proposal 1,Endorse,Education,0.000000,Mixed
4,Provide incentives for all teachers.,Revision/Addition,5,0,Proposal 2,Endorse,Education,0.000000,Mixed
5,Develop public campaigns and incentives for be...,Original,3,2,Proposal 1,Endorse,Healthcare,0.208333,Positive
6,Public PSA announcements reinstated,Revision/Addition,4,0,Proposal 1,Endorse,Healthcare,0.000000,Mixed
7,Develop policies/budgets to increase access to...,Revision/Addition,4,0,Proposal 1,Endorse,Healthcare,0.500000,Positive
8,"Ban Pharmaceutical ADs on Media radio, TV and ...",Revision/Addition,4,1,Proposal 1,Endorse,Healthcare,0.033333,Mixed
9,No financial or other incentives for docs to p...,Revision/Addition,4,0,Proposal 2,Endorse,Healthcare,-0.062500,Mixed


# Create a Google Spreadsheet and Quantatively Analyze

We will load the results from this notebook into a structured Google Sheet where the results can be stored for the future after the workshop.

This step will **require access to your Google credentials**, which you will need to get from Google Cloud. This file is the permsission that allows Jupyter notebooks to access your Google Drive files. For more details, [see this link](https://developers.google.com/workspace/guides/create-credentials).

In [ ]:
### Starting the Google Drive authentication process, this will give Jupyter notebooks
### Access to your Google Drive to create a spreadsheet and populate it with the data ###

!pip install --upgrade gspread pandas
from google.colab import auth
import gspread
from google.auth import default

# Authenticate Google account
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)


In [ ]:
### Creating a copy of the data to feed it into the spreadsheet with more intuitive columns ###

# Create a copy of the DataFrame with the specified columns
df_elements = pd.DataFrame({
    "Statement": df["Statement"],  # The original statements
    "Issue": df["Issue"],  # The original statements
    "Impacted Statement": df["Impacted Statement"],        # The assigned topic clusters
    "Type": df["Statement Type"],
    "Upvotes": df["Support Votes"],
    "Downvotes": df["Don’t Support Votes"],
    "Decision": df["Decision"],
    "Sentiment": df["Sentiment"],
    "Sentiment Score": df["Sentiment Score"]
})

# Display the new DataFrame
df_elements.head()


,Statement,Issue,Impacted Statement,Type,Upvotes,Downvotes,Decision,Sentiment,Sentiment Score
0,Revision: Incentivizing community colleges bas...,Economic Mobility,Proposal 1,Revision/Addition,0,0,Endorse,Mixed,0.0
1,Give lower-paid workers tax credits to increas...,Economic Mobility,Proposal 2,Original,0,0,Reject\r\n\tNote: This statement is a bit conf...,Mixed,0.0
2,Pay K-12 teachers more.,Education,Proposal 1,Revision/Addition,6,0,Endorse,Positive,0.5
3,"When determining “quality of education,” we sh...",Education,Proposal 1,Revision/Addition,6,0,Endorse,Mixed,0.0
4,Provide incentives for all teachers.,Education,Proposal 2,Revision/Addition,5,0,Endorse,Mixed,0.0


In [ ]:
# Google spreadsheet set uo
!pip install gspread gspread_dataframe oauth2client

In [ ]:
# Authenticate and connect to Google Sheets, YOU WILL NEED YOUR GOOGLE CREDENTIALS HERE
google_credentials_path = "ENTER YOUR FILE PATH"
scope = ["https://spreadsheets.google.com/feeds", 'https://www.googleapis.com/auth/drive']
creds = Credentials.from_service_account_file(google_credentials_path, scopes=scope)
client = gspread.authorize(creds)

In [ ]:
### Create the Google spreadsheet ###

import gspread
from gspread_dataframe import set_with_dataframe
from google.auth.transport.requests import Request
from google.oauth2.service_account import Credentials

#Add your ID
sheet_ID = "INSERT YOUR SHEET ID HERE"

# Define the spreadsheet URL and extract the ID
spreadsheet_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/edit"
spreadsheet_id = spreadsheet_url.split("/d/")[1].split("/")[0]

# Open the spreadsheet by ID
spreadsheet = client.open_by_key(spreadsheet_id)

# Function to clear a worksheet if it exists
def clear_sheet_if_exists(spreadsheet, sheet_name):
    try:
        worksheet = spreadsheet.worksheet(sheet_name)
        worksheet.clear()  # Clear the content of the worksheet
    except gspread.exceptions.WorksheetNotFound:
        # Sheet does not exist, no need to clear
        pass

# Clear existing "Elements" and "Connections" sheets if they already exist
clear_sheet_if_exists(spreadsheet, "Revised Framework")

# Add new worksheets for "Elements" and "Connections" only if they do not exist
try:
    elements_sheet = spreadsheet.worksheet("Revised Framework")
except gspread.exceptions.WorksheetNotFound:
    elements_sheet = spreadsheet.add_worksheet(title="Revised Framework", rows="1000", cols="20")

# Write data to the sheets
set_with_dataframe(elements_sheet, df_elements)

print(f"Data successfully written to Google Sheets! Take a look at: {spreadsheet_url}")


Data successfully written to Google Sheets!


In [ ]:
#Set up to use GenAI for summarization, you can use other GenAI libraries such as OpenAI
from google.colab import userdata
gemini_api = userdata.get('GEMINI_KEY')

import google.generativeai as genai
genai.configure(api_key=gemini_api)